In [ ]:
'''
CLEARING THE ARRAY AFTER EACH PREDICTION
'''
!pip install mediapipe
from cv2 import VideoCapture, waitKey, imshow, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, putText, FONT_HERSHEY_SIMPLEX, resize, rectangle, line
from numpy import ndarray, array, argmax, expand_dims, fromstring, zeros, concatenate
from pathlib import Path
from shutil import rmtree, move
from tensorflow.keras.models import load_model
from collections import deque
import time
import mediapipe as mp
from csv import writer
!nvidia-smi

     |████████████████████████████████| 32.7 MB 163 kB/s 
Thu Feb  3 06:55:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                              

In [ ]:
class MediapipeExtractor:
    def extract_landmarks(self, results) -> ndarray:
        result_landmarks = {
            'face': {
                'landmark': results.face_landmarks,
                'shape': (468,)
            },
            'left hand': {
                'landmark': results.left_hand_landmarks,
                'shape': (21,)
            },
            'right hand': {
                'landmark': results.right_hand_landmarks,
                'shape': (21,)
            },
            'pose': {
                'landmark': results.pose_landmarks,
                'shape': (33,)
            }
        }

        result_all = []
        result_temp = []
        for key, result in result_landmarks.items():
            if result['landmark']:
                result_temp = array([array([l.x, l.y, l.z]) for l in result['landmark'].landmark])
                result_temp = result_temp.flatten()
            else:
                result_temp = zeros((result['shape'][0] * 3, ))

            assert(result_temp.shape == (result['shape'][0] * 3, ))
            result_all.append(result_temp)
            
        shape = 0
        result_final = array([])
        for result in result_all:
            shape += result.shape[0]
            result_final = concatenate((result_final, result), axis=0)

        assert(result_final.shape == (shape,))

        return result_final

In [ ]:
class ImageHandler:
    def draw_results(self, image, results):
        image.flags.writeable = True

        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        return image

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
cap = VideoCapture(take_photo())
me = MediapipeExtractor()
im = ImageHandler()

# out_dir = Path('test')
# if out_dir.exists(): rmtree(out_dir)
# out_dir.mkdir()

<IPython.core.display.Javascript object>

TypeError: ignored

In [ ]:
QR = []
QR_limit = 50
QR_status = 0

#frame rate
prev_frame_time = 0
new_frame_time = 0
avgfps = 0

# model = r"Z:\jupyternotebook\ASL-main\ASL-main\test\Model\ADB_9_95"
# model = 'Model_new/ADB_9_95'
model = load_model('/content/drive/MyDrive/Model_new/ADB_9_95')

z=0
DQ = deque(maxlen=1)
Q = []
Q_limit = 70
Q_status = 0
klasses = ['Address', 'Movie', 'Name', 'Phone', 'Play', 'Please','Work','Your']

new_result = ""
new_result_1 = ""
text1 = ''
results_forever=""

In [ ]:
width = 640
height = 480

width_from_center = 175
height_from_center_top = 200
height_from_center_bottom = int(height / 2)

point1 = (int((width/2) - width_from_center), int((height/2) - height_from_center_top))
point2 = (int((width/2) + width_from_center), int((height/2) + height_from_center_bottom))

lheight = int((height/2) - 10)
lpoint1 = (point1[0], lheight)
lpoint2 = (point2[0], lheight)

fwidth = 100
fpoint1 = (int(width/2 - fwidth), point1[1])
fpoint2 = ((int(width/2 + fwidth), lpoint1[1]))


In [ ]:
with mp_holistic.Holistic(
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as holistic:

    while cap.isOpened():
        
        image: ndarray
        success, image = cap.read()

        if not success: continue

        image = flip(image, 1)

        original = image.copy()

        image = cvtColor(image, COLOR_BGR2RGB)

        image.flags.writeable = False
        results = holistic.process(image)

        image = cvtColor(image, COLOR_RGB2BGR)
        image = im.draw_results(image, results)

        landmarks = me.extract_landmarks(results)


        if QR_status == QR_limit:

            QR_vid = expand_dims(array(QR), 0)
            results = model.predict(QR_vid)

            # DQ.append(results[0])
            # results = array(DQ).mean(axis=0)
            results = array(results).mean(axis=0)
            predicted_kls = klasses[results.argmax(axis=0)]
            results_forever = results

            text1 = f"{predicted_kls}"
            QR.pop(0)
            QR.append(landmarks)
            z +=1
        else:
            QR.append(landmarks)
            QR_status += 1
            text2 = f"{'|'*QR_status}"
        
        image = resize(image, (1280, 1024))
        image = putText(image, str(text1), (30, image.shape[0] - 90), FONT_HERSHEY_SIMPLEX, 2, (11,209,252), 3)
        image = putText(image, text2, (30, image.shape[0] - 30), FONT_HERSHEY_SIMPLEX, 2, (11,209,252), 3)
        # new_result = (sorted(zip(klasses, results_forever), key=lambda x: x[1], reverse=True))
        z=0
        for x in sorted(zip(klasses, results_forever)):
            if z < 4:
                new_result = new_result + " " + str(x[0]) + " " + str(x[1]) + " "
                z +=1
            else :
                new_result_1 = new_result_1 + " " + str(x[0]) + " " + str(x[1]) + " "

        try:
            image = putText(image, str(new_result), (30, image.shape[0] - 980), FONT_HERSHEY_SIMPLEX, 0.7, (255,174,0), 2)
            image = putText(image, str(new_result_1), (30, image.shape[0] - 920), FONT_HERSHEY_SIMPLEX, 0.7, (255,174,0), 2)
            new_result = ""
            new_result_1 = ""
            
        except Exception as err:
            # pass
            # image = putText(image, str(results), (30, image.shape[0] - 200), FONT_HERSHEY_SIMPLEX, 1, (255,174,0), 2)
            print("hello")
        #fps counter

        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)
        avgfps=0.9*float(avgfps)+0.1*float(fps)
        original_text = str(original.shape) + "  FPS: " + str(int(avgfps))
        
        original = putText(original, original_text, (30, 30), FONT_HERSHEY_SIMPLEX, 1, (11,209,252), 2)
        original = rectangle(original, point1, point2, (211, 66, 242), 2)
        original = line(original, lpoint1, lpoint2, (211, 66, 242), 2)
        original = rectangle(original, fpoint1, fpoint2, (92, 206, 17), 2)
            
        imshow('out', image)
        imshow('original', original)

        if waitKey(5) & 0xFF == 27: break # 27 is escape!

    cap.release()

In [11]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
 
  return filename

In [ ]:
from IPython.display import Image
while True:
    take_photo()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>